# GAN Model | CIFAKE

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, LeakyReLU, Dropout, BatchNormalization, Reshape, UpSampling2D, Input
from keras.optimizers import Adam
from utils import load_model_history, load_augmented_data, build_sequential_model

In [2]:
model_name = "gan_model_cifake"
dataset_base_path = "./datasets/CIFAKE/"

# Forcefully train the model
force_model_train = False

# Don't save the model
no_save = False

In [3]:
# Load saved model and its history
model_history_loaded, classifier, history = load_model_history(model_name)

Model/history not found.


In [4]:
# Load augmented image generators
train_generator, validation_generator = load_augmented_data(
    dataset_base_path, batch_size=200)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
# Build generator
latent_dim = 100

if not model_history_loaded or force_model_train:
    generator = build_sequential_model(
        layers=[
            Dense(8 * 8 * 256, activation="relu", input_dim=latent_dim),
            Reshape((8, 8, 256)),
            UpSampling2D(),
            Conv2D(256, kernel_size=3, padding="same"),
            BatchNormalization(momentum=0.8),
            LeakyReLU(alpha=0.2),
            UpSampling2D(),
            Conv2D(128, kernel_size=3, padding="same"),
            BatchNormalization(momentum=0.8),
            LeakyReLU(alpha=0.2),
            Conv2D(64, kernel_size=3, padding="same"),
            BatchNormalization(momentum=0.8),
            LeakyReLU(alpha=0.2),
            Conv2D(3, kernel_size=3, padding="same", activation="tanh"),
        ],
        no_compile=True
    )

In [6]:
# Build discriminator
if not model_history_loaded or force_model_train:
    discriminator = build_sequential_model(
        layers=[
            Conv2D(32, kernel_size=3, strides=2, input_shape=(32,32,3), padding="same"),
            LeakyReLU(alpha=0.2),
            Dropout(0.25),
            Conv2D(64, kernel_size=3, strides=2, padding="same"),
            LeakyReLU(alpha=0.2),
            Dropout(0.25),
            Conv2D(128, kernel_size=3, strides=2, padding="same"),
            LeakyReLU(alpha=0.2),
            Dropout(0.25),
            Conv2D(256, kernel_size=3, strides=1, padding="same"),
            LeakyReLU(alpha=0.2),
            Dropout(0.25),
            Flatten(),
            Dense(1, activation='sigmoid'),
        ],
        optimizer=Adam(0.0002, 0.05)
    )


In [7]:
# Compile the GAN
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
img = generator(gan_input)
validity = discriminator(img)
gan = Model(gan_input, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [14]:
# Train GAN
def train_gan(generator, discriminator, gan, epochs, batch_size, latent_dim, train_generator):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        for real_imgs, _ in train_generator:
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator.predict(noise, verbose=0)
            
            d_loss_real = discriminator.train_on_batch(real_imgs, valid)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, valid)
            
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

train_gan(generator, discriminator, gan, epochs=100, batch_size=200, latent_dim=latent_dim, train_generator=train_generator)


KeyboardInterrupt: 